# Парсим 

In [1]:
import json
import os
import matplotlib as plt

In [2]:
texts_with_entities = []
for file in os.listdir('Разметка_все/'):
    if '.json' in file:
        with open(f'Разметка_все/{file}', 'r',  encoding="utf8") as json_data:
            json_text = json.load(json_data)
            
            for text_object in json_text:
                try: 
                    main_text = text_object['result'][list(text_object['result'].keys())[0]]['result']['marks'][0]['text']
                    ents = []
                    for entity_object in text_object['result'][list(text_object['result'].keys())[0]]['result']['marks'][1:]:

                        text = entity_object['text']
                        type_ = entity_object['entityId']
                        start = entity_object['position']['start']
                        end = entity_object['position']['end']

                        ents.append((text, type_, start, end))
                except: pass
                    
                texts_with_entities.append((main_text, ents))

In [4]:
len(texts_with_entities), sum([len(sample[1]) for sample in texts_with_entities])

(744, 35232)

# BIO

# e-obj и другие NE

In [6]:
from nltk.tokenize import sent_tokenize, word_tokenize, wordpunct_tokenize

In [7]:
text_to_sentences = []
for text, _ in texts_with_entities:
    
    sentences_bounds = {}
    start = 0
    
    for sentence in sent_tokenize(text, language='russian'):
        
        end = start + len(sentence)
        end += 1
        
        sentences_bounds[(start, end)] = sentence
        start = end
        
    text_to_sentences.append(sentences_bounds)

In [1]:
# text_to_sentences[0]

In [9]:
texts = [elem[0] for elem in texts_with_entities]
entities = [elem[1] for elem in texts_with_entities]

In [10]:
sum([len(text_to_sentences[i].keys()) for i in range(len(text_to_sentences))])

82361

In [11]:
sentences_with_ents = {}
c = 0
for i in range(len(texts)):
    current_doc_ents = entities[i]
    for sentence_borders in text_to_sentences[i].keys():
        for ent in current_doc_ents:
#             print(ent[2])
            if ent[2] > sentence_borders[0] and ent[3] < sentence_borders[1]:
                if (i, sentence_borders[0], sentence_borders[1]) in sentences_with_ents.keys():
                    sentences_with_ents[i, sentence_borders[0], sentence_borders[1]].append((ent[0], ent[1], ent[2], ent[3]))
                else:
                    sentences_with_ents[i, sentence_borders[0], sentence_borders[1]] = [(ent[0], ent[1], ent[2], ent[3])]

In [12]:
len(texts)

744

In [13]:
len(sentences_with_ents)

5370

In [15]:
def sentence_to_tokens(sentence, current_start):
    tokens = {}
    start_in_sequence = 0
    for token in wordpunct_tokenize(sentence):
        token_start = sentence[start_in_sequence:].find(token) + start_in_sequence + current_start
        token_end = token_start + len(token) - 1
        start_in_sequence = token_end - current_start
        tokens[token_start, token_end] = token  
    return tokens

def find_token(tokens, pos, mode):
    chosen_token = ''
    for token_start, token_end in tokens.keys():
        if token_start < pos < token_end:
            return tokens[token_start, token_end]
        elif token_end >= pos + 1 >= token_start and mode == 'start':
            return tokens[token_start, token_end]
        elif token_start <= pos - 1 <= token_end and mode == 'end':
            return tokens[token_start, token_end]
    
    return chosen_token
        

In [16]:
sentence = 'Мама мыла, раму!'
tokens = sentence_to_tokens(sentence, 100)
find_token(tokens, 104, mode='end'), tokens

('Мама',
 {(100, 103): 'Мама',
  (105, 108): 'мыла',
  (109, 109): ',',
  (111, 114): 'раму',
  (115, 115): '!'})

In [17]:
bio_samples = []
b = 0
e = 0
for text_idx, sent_start, sent_end in sentences_with_ents.keys():
    
    sentence = text_to_sentences[text_idx][(sent_start, sent_end)]
    tokens = sentence_to_tokens(sentence, sent_start)
    tokens_list = list(tokens.values())
    markup = ['O' for token in tokens_list]
    
    for entity in sentences_with_ents[(text_idx, sent_start, sent_end)]:
        
        start_token = find_token(tokens, entity[2], mode='start')
        end_token = find_token(tokens, entity[3], mode='end')
        
        if start_token == end_token and start_token != '':
            markup[tokens_list.index(start_token)] = f'i-{entity[1]}'
        elif start_token != '' and end_token != '':
            markup[tokens_list.index(start_token)] = f'i-{entity[1]}'
            markup[tokens_list.index(end_token)] = f'i-{entity[1]}'
            for i in range(tokens_list.index(start_token) + 1, tokens_list.index(end_token)):
                markup[i] = f'i-{entity[1]}'
            
        
    bio_samples.append((sentence, markup))
len(bio_samples)

5370

In [2]:
# wordpunct_tokenize(bio_samples[5][0]), bio_samples[5][1]

In [3]:
# bio_samples[1]

In [20]:
import pickle


In [21]:
with open('bio_all.pickle', 'wb') as file:
    pickle.dump(bio_samples, file)

In [22]:
with open('bio_all.pickle', 'rb') as file:
    bio = pickle.load(file)

In [23]:
text = []

for i in bio:
    text.append(i[0])

In [24]:
tag = []

for i in bio:
    tag.append(i[1])

In [25]:
list_tuples = list(zip(text, tag))

In [26]:
import pandas as pd 
data = pd.DataFrame(list_tuples, columns=['text', 'tag'])

In [27]:
data

,text,tag
0,Субсидии предоставляются в пределах бюджетных ...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1,"Основные понятия, используемые в настоящем Пор...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
2,Доступ к функциям сервиса для участников отбор...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
3,Отбор получателей субсидии проводится уполномо...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
4,Объявление о проведении отбора размещается на ...,"[O, O, O, i-e-treb-otbor, i-e-treb-otbor, O, O..."
...,...,...
5365,На дату подачи заявления претенденты должны со...,"[O, O, O, O, i-e-treb-rez, O, O, O, O, O, i-e-..."
5366,Настоящий Порядок определяет категории юридиче...,"[O, O, O, O, i-e-org-ul, i-e-org-ul, i-e-org-u..."
5367,В условиях настоящего Порядка используются сле...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
5368,Субсидированию подлежит часть фактически подтв...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [28]:
import nltk
import pymorphy2
!pip install pymorphy2-dicts

### Токенизация

In [29]:
from nltk.tokenize import wordpunct_tokenize
import numpy as np

In [30]:
MORPH = pymorphy2.MorphAnalyzer()

In [31]:
for i in range(len(data)):
    wrds = []
    for wrd in nltk.wordpunct_tokenize(data["text"][i]):
        wrd = MORPH.parse(wrd)[0].normal_form
        wrds.append(wrd)

    data.at[i, "text"] = wrds

In [32]:
data

,text,tag
0,"[субсидия, предоставляться, в, предел, бюджетн...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1,"[основной, понятие, ,, использовать, в, настоя...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
2,"[доступ, к, функция, сервис, для, участник, от...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
3,"[отбор, получатель, субсидия, проводиться, упо...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
4,"[объявление, о, проведение, отбор, размещаться...","[O, O, O, i-e-treb-otbor, i-e-treb-otbor, O, O..."
...,...,...
5365,"[на, дата, подача, заявление, претендент, долж...","[O, O, O, O, i-e-treb-rez, O, O, O, O, O, i-e-..."
5366,"[настоящий, порядок, определять, категория, юр...","[O, O, O, O, i-e-org-ul, i-e-org-ul, i-e-org-u..."
5367,"[в, условие, настоящий, порядок, использоватьс...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
5368,"[субсидирование, подлежать, часть, фактически,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


### Данные в числовой вид

In [33]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer

In [34]:
maxlen = max([len(s) for s in data["text"]])
maxlen

3295

In [35]:
tokenizer1 = Tokenizer(num_words=3295, filters="!»#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n")
# Токенайзер обновляет словарь токенов в соответствии с частотой вхождения слов.
tokenizer1.fit_on_texts(data["text"])

In [36]:
sequences1 = tokenizer1.texts_to_sequences(data["text"])

In [4]:
word_indexes = tokenizer1.word_index
# word_indexes

NameError: name 'tokenizer1' is not defined

In [38]:
tokenizer2 = Tokenizer(num_words=3295, filters="!»#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n")
# Токенайзер обновляет словарь токенов в соответствии с частотой вхождения слов.
tokenizer2.fit_on_texts(data["tag"])

In [39]:
sequences2 = tokenizer2.texts_to_sequences(data["tag"])

In [40]:
tag_indexes = tokenizer2.word_index
tag_indexes

{'o': 1,
 'i-e-treb-otbor': 2,
 'i-e-org-ul': 3,
 'i-e-v-selh': 4,
 'i-e-treb-nalog': 5,
 'i-e-treb-sub': 6,
 'i-e-treb-rez': 7,
 'i-e-v-selh-anim': 8,
 'i-e-treb-liq': 9,
 'i-e-prod-cred': 10,
 'i-e-v-transp': 11,
 'i-e-v-selh-rast': 12,
 'i-e-reg-msp': 13,
 'i-e-treb-dogov': 14,
 'i-e-org-ip': 15,
 'i-e-treb-dolg': 16,
 'i-e-prod-liz': 17,
 'i-e-obj': 18,
 'i-e-org-coop-spok': 19,
 'i-e-v-gkh': 20,
 'i-e-v-selh-anim-krs-plemya': 21,
 'i-e-org-kfl': 22,
 'i-e-v-transp-zaprav': 23,
 'i-e-treb-reg': 24,
 'i-e-reg-': 25,
 'i-e-org-sf': 26,
 'i-e-v-selh-rast-zerno': 27,
 'i-e-block-treb': 28,
 'i-e-v-selh-anim-krs-milk': 29,
 'i-e-v-transp-air': 30,
 'i-e-v-tur': 31,
 'i-e-prod-strax': 32,
 'i-e-org-lph': 33,
 'i-e-v-eda-eda': 34,
 'i-e-okved': 35,
 'i-e-v-selh-rast-kartof': 36,
 'i-e-v-eda': 37,
 'i-e-org-coop-coop': 38,
 'i-e-v-selh-anim-ovca': 39,
 'i-e-v-gkh-othod': 40,
 'i-e-v-selh-anim-bird': 41,
 'i-e-v-selh-fish': 42,
 'i-e-okved-main': 43,
 'i-e-v-selh-anim-krs': 44,
 'i-e-v-tran

In [41]:
n_token = len(list(set(word_indexes)))
n_token

6064

In [42]:
data["text_ind"] = list(map(list, data["text"]))

In [43]:
for i in range(len(data["text"])):
    data.at[i, "text_ind"] = sequences1[i]

In [44]:
data["tag_ind"] = list(map(list, data["tag"]))

In [45]:
for i in range(len(data["tag"])):
    data.at[i, "tag_ind"] = sequences2[i]

In [46]:
data

,text,tag,text_ind,tag_ind
0,"[субсидия, предоставляться, в, предел, бюджетн...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[10, 124, 2, 482, 83, 576, 3, 367, 83, 139, 1,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,"[основной, понятие, ,, использовать, в, настоя...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[856, 551, 1, 385, 2, 34, 13, 53, 171, 4, 35, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,"[доступ, к, функция, сервис, для, участник, от...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[1656, 50, 401, 2201, 37, 27, 11, 340, 1281, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,"[отбор, получатель, субсидия, проводиться, упо...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[11, 36, 10, 663, 327, 100, 564, 335, 171, 4, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,"[объявление, о, проведение, отбор, размещаться...","[O, O, O, i-e-treb-otbor, i-e-treb-otbor, O, O...","[114, 17, 41, 11, 536, 7, 158, 222, 83, 304, 2...","[1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
...,...,...,...,...
5365,"[на, дата, подача, заявление, претендент, долж...","[O, O, O, O, i-e-treb-rez, O, O, O, O, O, i-e-...","[7, 57, 89, 143, 445, 40, 141, 82, 60, 53, 12,...","[1, 1, 1, 1, 7, 1, 1, 1, 1, 1, 16, 16, 1, 1, 1..."
5366,"[настоящий, порядок, определять, категория, юр...","[O, O, O, O, i-e-org-ul, i-e-org-ul, i-e-org-u...","[34, 13, 252, 244, 22, 16, 4, 28, 79, 52, 4, 1...","[1, 1, 1, 1, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, ..."
5367,"[в, условие, настоящий, порядок, использоватьс...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[2, 77, 34, 13, 674, 82, 551, 53, 73, 25, 66, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
5368,"[субсидирование, подлежать, часть, фактически,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[1034, 774, 107, 673, 626, 73, 25, 66, 1, 456,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


#### Сохраним датасет

In [98]:
data.to_pickle('data_entities.pickle')

In [137]:
data = pd.read_pickle('data_entities.pickle')

In [100]:
with open('dict_tegs.pkl', 'wb') as f:
    pickle.dump(tag_indexes, f)

In [101]:
with open('dict_words.pkl', 'wb') as f:
    pickle.dump(word_indexes, f)